In [7]:
from librium.database import *
from csv import DictReader
import re
from sqlalchemy.orm.exc import NoResultFound

In [2]:
with open("books.csv") as fp:
    books = [x for x in DictReader(fp)]

In [9]:
for book in books:
    db_book = Book.query.filter(Book.isbn == book["isbn"], Book.title == book["title"]).one()
    
    # languages = [x.strip() for x in book["language"].split(",")]
    # for language in languages:
    #     db_lang = Language.query.filter(Language.name == language).one()
    #     db_book.languages.append(db_lang)
    # 
    # publishers = [x.strip() for x in book["publisher"].split(",")]
    # for publisher in publishers:
    #     db_pub = Publisher.query.filter(Publisher.name == publisher).one()
    #     db_book.publishers.append(db_pub)
    # 
    # genre = book["genre"]
    # db_genre = Genre.query.filter(Genre.name == genre).one_or_none()
    # if db_genre:
    #     db_book.genres.append(db_genre)
    # 
    # authors_raw = book["author_details"].split("|")
    # for author in authors_raw:
    #     _ = author.split(", ")
    #     f_author = " ".join(_[::-1]).strip()
    #     db_author = Author.query.filter(Author.name == f_author).one_or_none()
    #     db_book.authors.append(db_author)

    if book["series_details"]:
        series_pattern = re.compile(r"(?P<title>[^(]+)( ?\((?P<idx>[0-9.]+)\))?")
        series_raw = book["series_details"].split("|")
        for series in series_raw:
            srch = series_pattern.fullmatch(series)
            title = srch.group("title").strip()
            idx = float(srch.group("idx")) if srch.group("idx") else 0
            db_series = Series.query.filter(Series.name == title).one()
            try:
                db_series_index = SeriesIndex.query.filter(SeriesIndex.series == db_series).filter(SeriesIndex.book == db_book).one()
                if db_series_index.idx != idx:
                    db_series_index.idx = idx
            except NoResultFound:
                db_series_index = SeriesIndex(series=db_series, book=db_book, idx=idx)
                db_book.series.append(db_series_index)
            print(db_series_index)

    db_session.commit()

SeriesIndex(book_id=332, series_id=78, idx=1.0)
SeriesIndex(book_id=1059, series_id=78, idx=2.0)
SeriesIndex(book_id=928, series_id=78, idx=3.0)
SeriesIndex(book_id=1030, series_id=78, idx=4.0)
SeriesIndex(book_id=965, series_id=78, idx=5.0)
SeriesIndex(book_id=735, series_id=160, idx=1.0)
SeriesIndex(book_id=647, series_id=160, idx=2.0)
SeriesIndex(book_id=308, series_id=85, idx=3.0)
SeriesIndex(book_id=545, series_id=12, idx=2.0)
SeriesIndex(book_id=938, series_id=85, idx=5.0)
SeriesIndex(book_id=351, series_id=95, idx=1.0)
SeriesIndex(book_id=352, series_id=95, idx=2.0)
SeriesIndex(book_id=722, series_id=168, idx=1.0)
SeriesIndex(book_id=384, series_id=168, idx=2.0)
SeriesIndex(book_id=10, series_id=168, idx=3.0)
SeriesIndex(book_id=588, series_id=82, idx=1.0)
SeriesIndex(book_id=1037, series_id=82, idx=2.0)
SeriesIndex(book_id=834, series_id=82, idx=3.0)
SeriesIndex(book_id=744, series_id=82, idx=4.0)
SeriesIndex(book_id=363, series_id=82, idx=5.0)
SeriesIndex(book_id=756, series_i

MultipleResultsFound: Multiple rows were found for one()